# Embedding

In [5]:
# General libraries

import pandas as pd
import numpy as np

# Word2Vec

from gensim.models import Word2Vec 

# elmo

import tensorflow_hub as hub 
import tensorflow.compat.v1 as tf 
tf.disable_eager_execution() 

# Glove

import tensorflow

from tensorflow.keras.preprocessing.text import Tokenizer

# fasttext

import fasttext

2024-04-09 22:17:40.201085: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 22:17:40.203237: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 22:17:40.294666: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 22:17:40.688572: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 22:17:41.588240: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [6]:
data = pd.read_csv("preprocessed_data.csv")
data

,Unnamed: 0,News Articles,Unlike,Summary
0,0,think^real%danger%happens%data%cross%net%argue...,top%gig%award%scissor%sister^new%york%band%sci...,however%careful%may%organisation%trust%persona...
1,1,fast%moving%phone%virus%appear^security%firm%w...,black%sabbath%top%rock%album%poll^black%sabbat...,new%strain%cabir%mobile%phone%virus%use%shortr...
2,2,seaman%sail%biometric%future^luxury%cruise%lin...,farrell%due%make%u%tv%debut^actor%colin%farrel...,said%french%jordanian%nigerian%national%would%...
3,3,cable%offer%videoondemand^cable%firm%ntl%telew...,u%firm%bid%lacroix%label^u%firm%said%final%neg...,cable%firm%ntl%telewest%launched%videoondemand...
4,4,make%greener%computer^hitech%industry%starting...,star%pay%tribute%actor%davis^hollywood%star%in...,seeing%thing%technology%industry%result%billio...
...,...,...,...,...
2220,2220,circuit%city%get%takeover%offer^circuit%city%s...,saintandre%anger%absent%star^sale%shark%direct...,bill%armstrong%retail%analyst%cl%king%associat...
2221,2221,german%business%confidence%slide^german%busine...,mcconnell%drunk%remark%row^scotland%first%mini...,analyst%said%ifo%figure%germany%continuing%pro...
2222,2222,walmart%fight%back%accuser^two%big%u%name%laun...,ray%dvd%beat%box%office%taking^oscarnominated%...,meanwhile%drug%group%eli%lilly%planning%campai...
2223,2223,economy%stronger%forecast^uk%economy%probably%...,whitehall%cut%ahead%target^thousand%civil%serv...,mpc%judge%overall%growth%little%higher%third%q...


In [7]:
data.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   News Articles  2225 non-null   object
 1   Unlike         2225 non-null   object
 2   Summary        2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [9]:
def revert(sentence):
    x = sentence.split("^")
    return [i.split("%") for i in x]

In [10]:
data["News Articles"] = data["News Articles"].apply(revert)
data["Summary"] = data["Summary"].apply(revert)
data["Unlike"] = data["Unlike"].apply(revert)


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   News Articles  2225 non-null   object
 1   Unlike         2225 non-null   object
 2   Summary        2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [12]:
data.head()

,News Articles,Unlike,Summary
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo..."
4,"[[make, greener, computer], [hitech, industry,...","[[star, pay, tribute, actor, davis], [hollywoo...","[[seeing, thing, technology, industry, result,..."


In [13]:
def pad_words(l, sentence):
    n = ["_" for i in range(l-len(sentence))]
    if sentence == []:
        return n
    return n[:len(n)//2] + sentence + n[len(n)//2:]

In [14]:
def pad(text):
    max_sentence = 259
    max_word = 259
    l = max_sentence-len(text)
    f = [pad_words(max_word,[]) for i in range(l)]
    m = [pad_words(max_word,i) for i in text]
    return f[:len(f)//2]+m+f[len(f)//2:]

In [15]:
data["P News Articles"] = data["News Articles"].apply(pad)
data["P Summary"] = data["Summary"].apply(pad)
data["P Unlike"] = data["Unlike"].apply(pad)

In [16]:
data.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,..."
4,"[[make, greener, computer], [hitech, industry,...","[[star, pay, tribute, actor, davis], [hollywoo...","[[seeing, thing, technology, industry, result,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,..."


In [17]:
tmp = pd.DataFrame(data.iloc[:15,:])

## Note:

For the purposes of saving time and compute power, only the first ten rows have been used for all but the final embedding techniques. This is the usage of the BERT Transformer for embedding,

#### Vocabulary Creation

In [18]:
def create_vocabulary(text):
    words = []
    for i in text:
        words += i
    return set(words)

In [19]:
txt = []
for i in tmp.columns:
    for j in tmp[i]:
        s = []
        for k in j:
            s += k
        txt.append(s)
vocab = create_vocabulary(txt)
vocab

{'',
 'suggesting',
 'slum',
 'youd',
 'inciting',
 '98',
 'discorock',
 'surprising',
 'respect',
 'quantity',
 'computing',
 'engine',
 'achieve',
 'due',
 'singer',
 'coupon',
 'reported',
 'britishbacked',
 'alleged',
 'attallah',
 'properly',
 'documentary',
 'surrendering',
 'stepped',
 'unanimous',
 'funding',
 'including',
 'area',
 'radical',
 'faster',
 'dr',
 'challenge',
 'hair',
 'lightweight',
 '6ghz',
 'slattery',
 'latest',
 'shown',
 'february',
 'david',
 'list',
 '64',
 'handhelds',
 'pushing',
 'highend',
 'muse',
 'multitalented',
 'excellent',
 'earned',
 'online',
 'responding',
 'miramax',
 'station',
 'seat',
 'solo',
 'fsecure',
 '_',
 'priced',
 'florida',
 'malnourishment',
 'fax',
 'name',
 'kenteris',
 'paper',
 'inseparable',
 'notice',
 'counterpart',
 'miami',
 'booth',
 'surprised',
 '3d',
 'copied',
 'preacher',
 'manifesto',
 'becoming',
 'thefirst',
 'wellknown',
 'involved',
 'software',
 'unveils',
 'inadequate',
 'specified',
 'reserve',
 'edge',

### Bag of Words

In [20]:
def generate_bow(sentence):    
    bag_vector = np.zeros(len(vocab))
    for word in sentence: 
        for i,w in enumerate(vocab):
            if word == w: 
                bag_vector[i] += 1

    return np.array(bag_vector)[1:]

In [21]:
tmp["Bow News Articles"] = tmp["P News Articles"].apply(generate_bow)
tmp["Bow Summary"] = tmp["P Summary"].apply(generate_bow)
tmp["Bow Unlike"] = tmp["P Unlike"].apply(generate_bow)

In [22]:
tmp.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike,Bow News Articles,Bow Summary,Bow Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[[make, greener, computer], [hitech, industry,...","[[star, pay, tribute, actor, davis], [hollywoo...","[[seeing, thing, technology, industry, result,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### TF-IDF

In [23]:
def idf(texts):
    for i in vocab:
        n = 0
        for text in texts:
            if i in text:
                    n +=1
                    break
        encodings[i] = np.log(len(texts)/n)

In [24]:
encodings = {}
idf(txt)
encodings

{'': 4.499809670330265,
 'suggesting': 4.499809670330265,
 'slum': 4.499809670330265,
 'youd': 4.499809670330265,
 'inciting': 4.499809670330265,
 '98': 4.499809670330265,
 'discorock': 4.499809670330265,
 'surprising': 4.499809670330265,
 'respect': 4.499809670330265,
 'quantity': 4.499809670330265,
 'computing': 4.499809670330265,
 'engine': 4.499809670330265,
 'achieve': 4.499809670330265,
 'due': 4.499809670330265,
 'singer': 4.499809670330265,
 'coupon': 4.499809670330265,
 'reported': 4.499809670330265,
 'britishbacked': 4.499809670330265,
 'alleged': 4.499809670330265,
 'attallah': 4.499809670330265,
 'properly': 4.499809670330265,
 'documentary': 4.499809670330265,
 'surrendering': 4.499809670330265,
 'stepped': 4.499809670330265,
 'unanimous': 4.499809670330265,
 'funding': 4.499809670330265,
 'including': 4.499809670330265,
 'area': 4.499809670330265,
 'radical': 4.499809670330265,
 'faster': 4.499809670330265,
 'dr': 4.499809670330265,
 'challenge': 4.499809670330265,
 'hair

In [25]:
def tf(text,word):
    return text.count(word)/len(text)

In [26]:
def tfidf(text):
    encoded_text = []
    for i in text:
        x = []
        for j in i:
            encoded_text.append(tf(text,j)*encodings[j])
    return encoded_text

In [27]:
tmp["tfidf News Articles"] = tmp["P News Articles"].apply(tfidf)
tmp["tfidf Summary"] = tmp["P Summary"].apply(tfidf)
tmp["tfidf Unlike"] = tmp["P Unlike"].apply(tfidf)

In [28]:
tmp.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike,Bow News Articles,Bow Summary,Bow Unlike,tfidf News Articles,tfidf Summary,tfidf Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[[make, greener, computer], [hitech, industry,...","[[star, pay, tribute, actor, davis], [hollywoo...","[[seeing, thing, technology, industry, result,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Word2Vec

Skipgram

In [29]:
skipgram_model = Word2Vec(sentences=data['P News Articles'][0], vector_size=16, window=5, sg=0, min_count=1)

In [30]:
skipgram_model.wv.get_index("_")

0

In [31]:
skipgram_model.wv[0]

array([-0.19283263, -0.05265614,  0.39594948,  0.5589168 , -0.42480683,
       -0.48387256,  0.84819794,  0.5980617 , -0.2083466 , -0.12814385,
        0.06593548,  0.01661866, -0.109116  ,  0.12911186, -0.4637645 ,
       -0.2786901 ], dtype=float32)

In [32]:
def get_w2v_sg(text):
    x = []
    for i in text:
        x.append(skipgram_model.wv[int(i)])
    return x

In [33]:
tmp["w2v_sg News Articles"] = tmp["Bow News Articles"].apply(get_w2v_sg)
tmp["w2v_sg Summary"] = tmp["Bow Summary"].apply(get_w2v_sg)
tmp["w2v_sg Unlike"] = tmp["Bow Unlike"].apply(get_w2v_sg)

In [34]:
tmp.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike,Bow News Articles,Bow Summary,Bow Unlike,tfidf News Articles,tfidf Summary,tfidf Unlike,w2v_sg News Articles,w2v_sg Summary,w2v_sg Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589..."
4,"[[make, greener, computer], [hitech, industry,...","[[star, pay, tribute, actor, davis], [hollywoo...","[[seeing, thing, technology, industry, result,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.

Continuous Bag Of Words

In [35]:
cbow_model = Word2Vec(sentences=data['P News Articles'][0], vector_size=16, window=5, sg=1, min_count=1)

In [36]:
cbow_model.wv.get_index("_")

0

In [37]:
skipgram_model.wv[0]

array([-0.19283263, -0.05265614,  0.39594948,  0.5589168 , -0.42480683,
       -0.48387256,  0.84819794,  0.5980617 , -0.2083466 , -0.12814385,
        0.06593548,  0.01661866, -0.109116  ,  0.12911186, -0.4637645 ,
       -0.2786901 ], dtype=float32)

In [38]:
def get_w2v_cbow(text):
    x = []
    for i in text:
        x.append(cbow_model.wv[int(i)])
    return x

In [39]:
tmp["w2v_cbow News Articles"] = tmp["Bow News Articles"].apply(get_w2v_cbow)
tmp["w2v_cbow Summary"] = tmp["Bow Summary"].apply(get_w2v_cbow)
tmp["w2v_cbow Unlike"] = tmp["Bow Unlike"].apply(get_w2v_cbow)

tmp.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike,Bow News Articles,Bow Summary,Bow Unlike,tfidf News Articles,tfidf Summary,tfidf Unlike,w2v_sg News Articles,w2v_sg Summary,w2v_sg Unlike,w2v_cbow News Articles,w2v_cbow Summary,w2v_cbow Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2

### ELMo

In [40]:
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")
embeddings = embed(["A long sentence.", "single-word", "http://example.com"])
print(embeddings)


Tensor("keras_layer/StatefulPartitionedCall:0", shape=(None, 128), dtype=float32)


In [41]:
def elmo(text):
    w=[]
    for i in text:
        w.append(" ".join(i))
    embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")
    return embed(w)

In [42]:
tmp["elmo News Articles"] = tmp["News Articles"].apply(elmo)
tmp["elmo Summary"] = tmp["Summary"].apply(elmo)
tmp["elmo Unlike"] = tmp["Unlike"].apply(elmo)

tmp.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike,Bow News Articles,Bow Summary,Bow Unlike,tfidf News Articles,...,tfidf Unlike,w2v_sg News Articles,w2v_sg Summary,w2v_sg Unlike,w2v_cbow News Articles,w2v_cbow Summary,w2v_cbow Unlike,elmo News Articles,elmo Summary,elmo Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","Tensor(""keras_layer_1/StatefulPartitionedCall:...","Tensor(""keras_layer_16/StatefulPartitionedCall...","Tensor(""keras_layer_31/StatefulPartitionedCall..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","Tensor(""keras_layer_2/StatefulPartitionedCall:...","Tensor(""keras_layer_17/StatefulPartitionedCall...","Tensor(""keras_layer_32/StatefulPartitionedCall..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","Tensor(""keras_layer_3/StatefulPartitionedCall:...","Tensor(""keras_layer_18/StatefulPartitionedCall...","Tensor(""keras_layer_33/StatefulPartitionedCall..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

### Glove

In [43]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(vocab)

In [45]:
tokenizer.word_index["journey"]

1387

In [48]:
tokenizer.texts_to_matrix([["hello journey"],["life journey"]])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
def glove(text):
    w=[]
    for i in text:
        w.append(" ".join(i))
    return tokenizer.texts_to_matrix(w)

In [52]:
tmp["glove News Articles"] = tmp["News Articles"].apply(glove)
tmp["glove Summary"] = tmp["Summary"].apply(glove)
tmp["glove Unlike"] = tmp["Unlike"].apply(glove)

tmp.head()

,News Articles,Unlike,Summary,P News Articles,P Summary,P Unlike,Bow News Articles,Bow Summary,Bow Unlike,tfidf News Articles,...,w2v_sg Unlike,w2v_cbow News Articles,w2v_cbow Summary,w2v_cbow Unlike,elmo News Articles,elmo Summary,elmo Unlike,glove News Articles,glove Summary,glove Unlike
0,"[[think], [real, danger, happens, data, cross,...","[[top, gig, award, scissor, sister], [new, yor...","[[however, careful, may, organisation, trust, ...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","Tensor(""keras_layer_1/StatefulPartitionedCall:...","Tensor(""keras_layer_16/StatefulPartitionedCall...","Tensor(""keras_layer_31/StatefulPartitionedCall...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,"[[fast, moving, phone, virus, appear], [securi...","[[black, sabbath, top, rock, album, poll], [bl...","[[new, strain, cabir, mobile, phone, virus, us...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","Tensor(""keras_layer_2/StatefulPartitionedCall:...","Tensor(""keras_layer_17/StatefulPartitionedCall...","Tensor(""keras_layer_32/StatefulPartitionedCall...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,"[[seaman, sail, biometric, future], [luxury, c...","[[farrell, due, make, u, tv, debut], [actor, c...","[[said, french, jordanian, nigerian, national,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[[-0.19283263, -0.05265614, 0.39594948, 0.5589...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","[[-0.15329507, 0.070248745, 1.4148641, 2.50316...","Tensor(""keras_layer_3/StatefulPartitionedCall:...","Tensor(""keras_layer_18/StatefulPartitionedCall...","Tensor(""keras_layer_33/StatefulPartitionedCall...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,"[[cable, offer, videoondemand], [cable, firm, ...","[[u, firm, bid, lacroix, label], [u, firm, sai...","[[cable, firm, ntl, telewest, launched, videoo...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[[-0.19283263, -0.05265614, 0.39594

### Fasttext

In [53]:
tmp = tmp[["News Articles", "Summary", "Unlike"]]

In [54]:
x = ""
for i in tmp.columns:
    s = []
    for j in tmp[i]:
        f = []
        for k in j:
            f.append(" ".join(k))
        s.append(" ".join(f))
    x += " ".join(s)
x

'think real danger happens data cross net argues analyst bill thompson  happens arrives end financial service authority warned bank financial institution member criminal gang may applying job give access confidential customer data  fear steal money bank account instead steal something far valuable digital society identity  armed personal detail bank hold plus fake letter two apparently easy get loan open bank account overdraft get credit card someone el name  simple matter move money another account leave unwitting victim sort mess statement demand payment start arriving  identity theft increasingly significant economic crime becoming aware danger leaving bill receipt bank statement unshredded rubbish however careful may organisation trust personal data bank account credit card able look database properly trouble  surprising taken gang long realise wellplaced insider far simplest way break security computer system  fact suspect fsa probably late particular party sort thing going rather

In [55]:
with open("fastext_data.txt","w") as f:
    f.write(x)

In [56]:
smodel = fasttext.train_unsupervised('fastext_data.txt', model='skipgram')

Read 0M words
Number of words:  499
Number of labels: 0
Progress: 100.0% words/sec/thread:   38375 lr:  0.000000 avg.loss:  2.960549 ETA:   0h 0m 0s


In [57]:
cmodel = fasttext.train_unsupervised('fastext_data.txt', model='cbow')

Read 0M words
Number of words:  499
Number of labels: 0
Progress: 100.0% words/sec/thread:   78827 lr:  0.000000 avg.loss:  3.719630 ETA:   0h 0m 0s


In [58]:
cmodel["Ambidextrous"]

array([ 1.43614355e-02, -3.41421342e-03,  5.59345027e-03,  3.17057595e-02,
       -1.82644408e-02, -4.22105156e-02,  1.62673928e-02,  4.30237427e-02,
        3.61539014e-02,  2.08353810e-02, -2.89062094e-02,  3.04791927e-02,
        8.04490522e-02, -2.27557663e-02,  4.72512469e-02,  3.72148827e-02,
        4.29939106e-03, -2.71447841e-02,  4.87840362e-02, -3.29619646e-02,
       -2.28897799e-02, -3.05391271e-02,  2.67198253e-02, -6.32210076e-03,
       -9.85555798e-02,  2.80325040e-02, -3.25257108e-02, -2.35312879e-02,
       -5.56619987e-02,  8.27566683e-02, -3.61502031e-03, -2.80707683e-02,
        1.86931361e-02, -1.89545304e-02, -1.00198193e-02,  1.12013565e-02,
        8.33546743e-03, -7.60567784e-02,  3.16670127e-02,  3.94066460e-02,
        1.31294973e-05, -3.59119177e-02, -4.61134426e-02,  1.57825295e-02,
       -7.21660182e-02, -1.78893730e-02,  1.63497869e-02,  1.05607118e-02,
       -3.80591340e-02,  6.06410615e-02,  2.90060807e-02,  4.88418341e-02,
        1.05880732e-02,  